In [146]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X = pd.read_csv("../data/bank.csv")
X.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [147]:
X.shape
X.isnull().sum().sum() # 2 volte sum per comprimere la matrice

np.int64(0)

# 2 - Converti i valori di Gender in numerici (Male = 0, Female = 1).

In [148]:
gender = {'Male': 0, 'Female': 1}
X['Gender'] = X['Gender'].map(gender)
X.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,1,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,1,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,1,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,1,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,1,43,2,125510.82,1,1,1,79084.10,0


# 3 - Trasforma la feature Geography con OneHotEncoder.

In [149]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo = np.array(X['Geography']).reshape(-1, 1)
sparse = ohe.fit_transform(geo)
X = pd.concat([X, pd.DataFrame.sparse.from_spmatrix(sparse)], axis=1)
X.rename(columns={0:'France', 2:'Spain', 1:'Germany'}, inplace = True)
X.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,1,15634602,Hargrave,619,France,1,42,2,0.00,1,1,1,101348.88,1,1.0,0,0
1,2,15647311,Hill,608,Spain,1,41,1,83807.86,1,0,1,112542.58,0,0,0,1.0
2,3,15619304,Onio,502,France,1,42,8,159660.80,3,1,0,113931.57,1,1.0,0,0
3,4,15701354,Boni,699,France,1,39,1,0.00,2,0,0,93826.63,0,1.0,0,0
4,5,15737888,Mitchell,850,Spain,1,43,2,125510.82,1,1,1,79084.10,0,0,0,1.0


# 4 -
Elimina le feature RowNumber, CustomerId, Surname e dividi il dataset in train (75% del
dataset) e test (25%). Usa random_state=0.

In [150]:
X = X.drop(['RowNumber', 'CustomerId', 'Surname', 'Geography'], axis=1)
X_train , X_test, y_train, y_test = train_test_split(X.drop('Exited', axis=1), X['Exited'], test_size = 0.25, random_state = 0)
X_train.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
2967,579,1,39,5,117833.30,3,0,0,5831.00,0,1.0,0
700,750,1,32,5,0.00,2,1,0,95611.47,1.0,0,0
3481,729,1,34,9,53299.96,2,1,1,42855.97,0,0,1.0
1621,689,0,38,5,75075.14,1,1,1,8651.92,0,0,1.0
800,605,0,52,7,0.00,2,1,1,173952.50,1.0,0,0


In [151]:
y_train.head(5)

2967    1
700     0
3481    0
1621    1
800     0
Name: Exited, dtype: int64

# 5 -
Allena un decision tree per predire l’abbandono dei clienti, scegli tu i valori dei parametri e
calcola l’accuratezza della predizione.

In [152]:
from sklearn.tree import DecisionTreeClassifier
dst = DecisionTreeClassifier()
dst.fit(X_train, y_train)
accuracy_score(y_test, dst.predict(X_test))

/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/utils/validation.py:919: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/utils/validation.py:919: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.7948

# 6 -
Aggiungi una nuova feature nel dataset con il valore di (EstimatedSalary*Tenure+Balance)/2.
L’accuratezza della predizione migliora?

In [173]:
X['help'] = (X['EstimatedSalary'] * X['Tenure'] + X['Balance']) / 2
X_train , X_test, y_train, y_test = train_test_split(X.drop('Exited', axis=1), X['Exited'], test_size = 0.25, random_state = 0)

X_train.head(5)

dst = DecisionTreeClassifier()
dst.fit(X_train, y_train)
accuracy_score(y_test, dst.predict(X_test))


/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/utils/validation.py:919: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/utils/validation.py:919: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.7996

# 7 -
Applica due trasformazioni diverse a tutto il dataset e/o a feature specifiche, valuta la predizione.

In [178]:
import sklearn.preprocessing as preprocessing

balance_norm = pd.DataFrame(preprocessing.normalize(X['Balance'].values.reshape(-1,1), norm='l2'), columns=['NormBalance'])
esalary_norm = pd.DataFrame(preprocessing.normalize(X['EstimatedSalary'].values.reshape(-1,1), norm='l2'), columns=['NormEstimatedSalary'])

X_norm = pd.concat([X.drop(['Balance', 'EstimatedSalary'], axis=1), balance_norm, esalary_norm], axis=1)
X_train , X_test, y_train, y_test = train_test_split(X_norm.drop('Exited', axis=1), X['Exited'], test_size = 0.25, random_state = 0)



dst = DecisionTreeClassifier()
dst.fit(X_train, y_train)
accuracy_score(y_test, dst.predict(X_test))


/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/utils/validation.py:919: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/utils/validation.py:919: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.7864